In [91]:
import pandas as pd
import numpy as np
from pathlib import Path

# ---------------- CONFIG ----------------
ID_COL   = "DW_EK_Borger"
WIN_PATH = Path(r"C:\Users\kfq6\Documents\Data\Bookplan_sammedag_month_windows.xlsx")
POP_PATH = Path(r"C:\Users\kfq6\Documents\Data\Population.xlsx")  # <- add this
WHO_PATH = Path(r"C:\Users\kfq6\Documents\Data\WHO-5 (PRO).xlsx")
LAB_PATH = Path(r"C:\Users\kfq6\Documents\Data\LABKA_wide_rawSvar.xlsx")
OUT_PATH = Path(r"C:\Users\kfq6\Documents\Data\Sammedag_master_means.xlsx")

STRICT_NUMERIC_ANALYTES = True  # keep only analyte columns that are actually numeric after coercion


In [92]:
# ---------------- HELPERS ----------------
def ensure_dates(df: pd.DataFrame, cols):
    for c in cols:
        if c not in df.columns:
            raise KeyError(f"Missing required datetime column: {c}")
        df[c] = pd.to_datetime(df[c], errors="coerce", dayfirst=True)
    return df

def pick_closest_in_window(df: pd.DataFrame, date_col: str, anchor: pd.Timestamp, start: pd.Timestamp, end: pd.Timestamp):
    sub = df[(df[date_col] >= start) & (df[date_col] <= end)].copy()
    if sub.empty:
        return None
    sub["abs_days_from_anchor"] = (sub[date_col] - anchor).abs().dt.days
    sub = sub.sort_values(["abs_days_from_anchor", date_col], ascending=[True, False])
    return sub.iloc[0:1]

def month_window(anchor: pd.Timestamp):
    return anchor - pd.DateOffset(months=1), anchor + pd.DateOffset(months=1)

def coerce_numeric(df: pd.DataFrame, cols):
    out = {}
    for c in cols:
        s = (
            df[c].astype(str)
                 .str.replace(",", ".", regex=False)
                 .replace({"nan": None, "None": None, "": None})
        )
        out[c] = pd.to_numeric(s, errors="coerce")
    return pd.DataFrame(out, index=df.index)


In [93]:
# ---------------- LOAD WINDOWS ----------------
windows = pd.read_excel(
    WIN_PATH, engine="openpyxl",
    parse_dates=["anchor_date", "window_start", "window_end"]
)
need_win = {ID_COL, "anchor_date", "window_start", "window_end"}
missing = need_win - set(windows.columns)
if missing:
    raise KeyError(f"'windows' missing columns: {missing}")

In [94]:
# ---------------- LOAD WHO ----------------
who = pd.read_excel(WHO_PATH, engine="openpyxl")
who.columns = who.columns.str.strip()

WHO_DATE_CANDIDATES = ["Dato_Besoeg", "Dato_Spoergeskema_Oprettelse", "Dato_Visit", "Dato"]
who_date_col = next((c for c in WHO_DATE_CANDIDATES if c in who.columns), None)
if not who_date_col:
    raise KeyError(f"WHO file missing a date column; expected one of: {', '.join(WHO_DATE_CANDIDATES)}")
who = ensure_dates(who, [who_date_col])
if ID_COL not in who.columns:
    raise KeyError(f"WHO file missing ID column: {ID_COL}")

WHO_SCORE_CANDIDATES = ["WHO5_score", "WHO5", "WHO-5", "WHO5_total"]
who_score_col = next((c for c in WHO_SCORE_CANDIDATES if c in who.columns), None)




In [95]:
import datetime as dt

# ---------------- LOAD POPULATION ----------------
pop = pd.read_excel(POP_PATH, engine="openpyxl")

# Basic guards
if ID_COL not in pop.columns:
    raise KeyError(f"Population file missing ID column: {ID_COL}")
if "Aktionsdiagnosegruppe" not in pop.columns:
    raise KeyError("Population file missing column: 'Aktionsdiagnosegruppe'")
if "CPRNummer" not in pop.columns:
    raise KeyError("Population file missing CPRNummer column")

# Clean diagnosis text
pop["Aktionsdiagnosegruppe"] = (
    pop["Aktionsdiagnosegruppe"]
      .astype(str).str.strip()
      .replace({"": np.nan, "nan": np.nan, "None": np.nan})
)

def pick_mode(series: pd.Series):
    s = series.dropna()
    if s.empty:
        return np.nan
    m = s.mode()  # may be multiple
    return m.sort_values().iloc[0]

# --- Clean CPR numbers (strip separators, left-pad) ---
CPR_COL = "CPRNummer"
pop[CPR_COL] = (
    pop[CPR_COL]
      .astype(str)
      .str.replace(r"\D", "", regex=True)
      .str.zfill(10)
)

# --- Decode birthdate from DDMMYY with a sane century heuristic ---
def parse_cpr_birth(cpr: str):
    if not isinstance(cpr, str) or len(cpr) < 6:
        return np.nan
    try:
        dd, mm, yy = int(cpr[:2]), int(cpr[2:4]), int(cpr[4:6])
        today = dt.date.today()
        century = 1900 if yy > today.year % 100 else 2000
        return dt.date(century + yy, mm, dd)
    except Exception:
        return np.nan

pop["birth_date"] = pop[CPR_COL].apply(parse_cpr_birth)

# --- Decode sex from LAST digit (even=F, odd=M) ---
def parse_cpr_sex(cpr: str):
    if not isinstance(cpr, str) or len(cpr) < 10:
        return np.nan
    try:
        return "M" if int(cpr[-1]) % 2 == 1 else "F"
    except Exception:
        return np.nan

pop["sex"] = pop[CPR_COL].apply(parse_cpr_sex)

# --- Age in years (rounded to 1 decimal) ---
today_ts = pd.Timestamp.today().normalize()
pop["age"] = ((today_ts - pd.to_datetime(pop["birth_date"], errors="coerce")).dt.days / 365.25).round(1)

# --- One row per patient: diagnosis (mode), sex (mode), age (mean) ---
pop_diag = (
    pop.groupby(ID_COL, as_index=False)
       .agg({
           "Aktionsdiagnosegruppe": pick_mode,
           "sex": pick_mode,
           "age": "mean"
       })
       .rename(columns={"Aktionsdiagnosegruppe": "diagnosis"})
       [[ID_COL, "sex", "age", "diagnosis"]]
)

print(pop_diag.head())

   DW_EK_Borger sex   age        diagnosis
0          2611   M  59.8  Type 2-diabetes
1          2822   M  55.8  Type 1-diabetes
2          2897   M  54.8  Type 2-diabetes
3          3557   M  43.8  Type 1-diabetes
4          4001   M  37.8  Type 1-diabetes


In [96]:
# ---------- Load LABKA (wide) ----------
lab = pd.read_excel(LAB_PATH, engine="openpyxl")

# --- Decode Excel-style XML escapes in strings (values and column names) ---
def decode_excel_xml(s):
    if not isinstance(s, str):
        return s
    return re.sub(r"_x([0-9A-Fa-f]{4})_", lambda m: chr(int(m.group(1), 16)), s)

# Decode column names (some headers have escapes too)
lab.rename(columns=lambda c: decode_excel_xml(c) if isinstance(c, str) else c, inplace=True)

# Identify the LAB date column
LAB_DATE_CANDIDATES = ["Testdato", "Dato_Proevetagningstid", "Prøvetagningstid", "Dato"]
lab_date_col = next((c for c in LAB_DATE_CANDIDATES if c in lab.columns), None)
if not lab_date_col:
    raise KeyError(f"LAB file missing a date column; expected one of: {', '.join(LAB_DATE_CANDIDATES)}")

# Make sure core columns exist and are datetimes
if ID_COL not in lab.columns:
    raise KeyError(f"LAB file missing ID column: {ID_COL}")
lab[lab_date_col] = pd.to_datetime(lab[lab_date_col], errors="coerce")

# List of analyte columns to coerce (same headers you showed)
orig_cols = [
    'B-Hæmoglobin_[mmol_x002F_l]',
    'Hb_x0028_B_x0029_-Hæmoglobin_x0020_A1c_x0020__x0028_IFCC_x0029__[mmol_x002F_mol]',
    'P-25-Hydroxy-Vitamin_x0020_D_x0028_D3_x002B_D2_x0029__[nmol_x002F_l]',
    'P-Albumin_[g_x002F_l]',
    'P-Calcium_[mmol_x002F_l]',
    'P-Calcium_x0020__x0028_albuminkorrigeret_x0029__[mmol_x002F_l]',
    'P-Kalium_[mmol_x002F_l]',
    'P-Kolesterol_[mmol_x002F_l]',
    'P-Kolesterol_x0020_HDL_[mmol_x002F_l]',
    'P-Kolesterol_x0020_LDL_[mmol_x002F_l]',
    'P-Kreatinin_[_x00B5_mol_x002F_l]',
    'P-Natrium_[mmol_x002F_l]',
    'P-Triglycerid_[mmol_x002F_l]',
    'P-Vitamin_x0020_B12_[pmol_x002F_l]',
    'U-Albumin_x0020__x002F__x0020_Kreatinin-ratio_[_x00D7__x0020_10_x003C_sup_x003E_-3_x003C__x002F_sup_x003E_]',
    'U-Albumin_x0020__x002F__x0020_Kreatinin-ratio_[mg_x002F_g]',
    'eGFR_x0020__x002F__x0020_1_x002C_73m_x00B2__x0028_CKD-EPI_x0029__[ml_x002F_min]'
]

# Column names might decode; map them through the same decoder and keep only what's present
cols_to_float = [decode_excel_xml(c) for c in orig_cols]
cols_to_float = [c for c in cols_to_float if c in lab.columns]

# Convert: decode cell text -> handle >/< -> comma to dot -> strip -> to_numeric
for c in cols_to_float:
    lab[c] = (
        lab[c]
        .map(decode_excel_xml)                     # decode _xHHHH_ inside cell values
        .astype(str)
        .str.replace(r'^\s*[<>]=?\s*', '', regex=True)  # drop leading >, >=, <, <=
        .str.replace(',', '.', regex=False)              # Danish decimal comma
        .str.replace('\u00A0', '', regex=False)          # non-breaking spaces, just in case
        .str.replace('−', '-', regex=False)              # unicode minus
        .str.strip()
        .pipe(pd.to_numeric, errors='coerce')
    )

# Optionally restrict to strictly numeric analytes if you set the flag
if STRICT_NUMERIC_ANALYTES:
    lab_value_cols = [c for c in cols_to_float if pd.api.types.is_numeric_dtype(lab[c])]
else:
    lab_value_cols = cols_to_float

# Quick sanity peek
print("LAB date column:", lab_date_col)
print("Analytes to mean:", len(lab_value_cols))


LAB date column: Testdato
Analytes to mean: 17


In [97]:
# ---------------- BUILD MASTER (MEANS ONLY) ----------------
rows = []
carry_cols = ["n_visits_in_window", "visit_datetimes", "visit_topics", "visit_statuses"]  # will fill if present

for w in windows.itertuples(index=False):
    wd = w._asdict()
    pid = wd[ID_COL]
    anchor = pd.to_datetime(wd["anchor_date"])
    win_start = pd.to_datetime(wd["window_start"])
    win_end   = pd.to_datetime(wd["window_end"])

    # WHO: closest inside anchor's window
    who_pid  = who[who[ID_COL] == pid]
    who_pick = pick_closest_in_window(who_pid, who_date_col, anchor, win_start, win_end)
    who_present = who_pick is not None
    who_date = who_pick[who_date_col].iloc[0] if who_present else pd.NaT
    who_days = (who_date - anchor).days if who_present else None
    who_score = (
        who_pick[who_score_col].iloc[0]
        if (who_present and who_score_col and who_score_col in who_pick.columns)
        else None
    )

    # LAB: mean over ±1 month window around the anchor
    lab_pid = lab[lab[ID_COL] == pid]
    lab_ws, lab_we = month_window(anchor)
    sub = lab_pid[(lab_pid[lab_date_col] >= lab_ws) & (lab_pid[lab_date_col] <= lab_we)].copy()

    lab_present = not sub.empty
    lab_means = {}
    if lab_present and lab_value_cols:
        means = sub[lab_value_cols].mean(axis=0, skipna=True)
        lab_means = {f"LABmean__{c}": means[c] for c in lab_value_cols}

    row = {
        ID_COL: pid,
        "anchor_date": anchor,
        "window_start": win_start,
        "window_end": win_end,
        # WHO
        "has_who_in_window": bool(who_present),
        "who_date": who_date,
        "who_days_from_anchor": who_days,
        "who_score": who_score,
        # LAB
        "has_lab_in_window": bool(lab_present),
        "lab_window_start": lab_ws,
        "lab_window_end": lab_we,
        "lab_window_n_rows": int(sub.shape[0]) if lab_present else 0,
    }
    for c in carry_cols:
        row[c] = wd.get(c)
    row.update(lab_means)
    rows.append(row)

master_means = pd.DataFrame(rows)




In [98]:
# --- Merge Population first (bring in diagnosis + sex + age) ---
cols_from_pop = [ID_COL, "diagnosis", "sex", "age"]
pop_view = pop_diag[[c for c in cols_from_pop if c in pop_diag.columns]]
master_means = master_means.merge(pop_view, on=ID_COL, how="left")

# --- Order columns: ID + sex/age/diagnosis, then dates/flags, then leftovers, then LAB means ---
front = [
    ID_COL, "sex", "age", "diagnosis",
    "anchor_date", "window_start", "window_end",
    "n_visits_in_window", "visit_datetimes", "visit_topics", "visit_statuses",
    "has_who_in_window", "who_date", "who_days_from_anchor", "who_score",
    "has_lab_in_window", "lab_window_start", "lab_window_end", "lab_window_n_rows",
]

# All LAB means at the end, sorted
mean_cols = sorted([c for c in master_means.columns if c.startswith("LABmean__")])

# Keep anything else that snuck in (but not duplicates of front/means)
leftovers = [c for c in master_means.columns if c not in set(front + mean_cols)]

# Final ordered list, only keep columns that actually exist
ordered_cols = [c for c in front if c in master_means.columns] + leftovers + mean_cols
master_means = master_means[ordered_cols]


In [99]:
# Single output
master_means.to_excel(OUT_PATH, index=False)
print(f"Saved single dataset with LAB means only: {OUT_PATH}")

# Quick stats (optional sanity)
n_all = len(master_means)
n_complete = int(((master_means["has_who_in_window"]) & (master_means["has_lab_in_window"])).sum())
print(f"Complete anchors (WHO+LAB present): {n_complete}/{n_all} "
      f"({(n_complete/n_all*100 if n_all else 0):.1f}%)")

Saved single dataset with LAB means only: C:\Users\kfq6\Documents\Data\Sammedag_master_means.xlsx
Complete anchors (WHO+LAB present): 2736/4597 (59.5%)
